In [22]:
import pandas as pd

In [23]:
# Load datasets: one with Florida census tract (CT) 10-digit IDs, another with environmental variables and 11-digit IDs
df1 = pd.read_csv('Florida_ct.csv')  # Contains 10-digit IDs representing census tracts in Florida
df2 = pd.read_excel('EPA_SmartLocationDatabase_V3_Jan_2021_Final.xlsx')  # Contains environmental data with 11-digit IDs

In [24]:
# Truncate the 11-digit IDs in df2 to create a new column for matching with df1's 10-digit IDs
df2['truncated_id'] = df2['GEOID20'].astype(str).str[:6]
print(df2.columns)
matches = df1['full_ct_fips'].astype(str).str[:6].isin(df2['truncated_id'])
print("Number of matching IDs:", matches.sum())
df1['full_ct_fips'] = df1['full_ct_fips'].astype(str)
df1['match_id'] = df1['full_ct_fips'].str[:6]

Index(['OBJECTID', 'GEOID10', 'GEOID20', 'STATEFP', 'COUNTYFP', 'TRACTCE',
       'BLKGRPCE', 'CSA', 'CSA_Name', 'CBSA',
       ...
       'D5DE', 'D5DEI', 'D2A_Ranked', 'D2B_Ranked', 'D3B_Ranked', 'D4A_Ranked',
       'NatWalkInd', 'Shape_Length', 'Shape_Area', 'truncated_id'],
      dtype='object', length=118)
Number of matching IDs: 3660


In [25]:
value_columns = ['D1B', 'D1C','D5AR', 'D2R_JOBPOP', 'D2B_E5MIXA', 'D2B_E8MIXA', 'D2A_EPHHM', 'D2C_TRIPEQ', 'D1C5_RET', 'D1C5_IND', 'D1C5_SVC', 'D1C5_ENT', 'D3A', 'D3BPO3', 'D3B', 'D3BAO', 'D3BMM3', 'D3BMM4', 'D3BPO4']

In [26]:
# Group by the truncated_id and calculate the mean for each specified column
average_values = df2.groupby('truncated_id')[value_columns].mean().reset_index()
df1['full_ct_fips'] = df1['full_ct_fips'].astype(str)

In [27]:
# Merge the average values back to df1 based on the ID
result_df = pd.merge(df1, average_values, left_on='match_id', right_on='truncated_id', how='left')

In [28]:
# Drop the 'truncated_id' column if it's no longer needed
result_df.drop(columns=['match_id', 'truncated_id'], inplace=True)

In [29]:
columns_rename_map = {
    'D1B': 'population_density',
    'D1C': 'employment_density',
    'D5AR': 'accessibility_index',
    'D2R_JOBPOP': 'regional_diversity',
    'D2B_E5MIXA': '5-tier_employment_entropy',
    'D2B_E8MIXA': '8-tier_employment_entropy',
    'D2A_EPHHM': 'employment_household_entropy',
    'D2C_TRIPEQ': 'trip_equilibrium_index',
    'D1C5_RET': 'retail_employment_density',
    'D1C5_IND': 'industrial_employment_density',
    'D1C5_SVC': 'service_employment_density',
    'D1C5_ENT': 'entertainment_employment_density',
    'D3A': 'road_network_density',
    'D3BPO3': 'pedestrian_intersection_density',
    'D3B': 'street_intersection_density',
    'D3BAO': 'auto_intersection_density',
    'D3BMM3': 'multi-modal_intersection_density',
    'D3BMM4': 'multi-modal_intersection_density(>4legs)',
    'D3BPO4': 'pedestrian_intersection_density(>4legs)'
}

result_df = result_df.rename(columns=columns_rename_map)

In [30]:
result_df.to_csv('Merged_Florida_ct.csv', index=False)